In [250]:
import zipfile
from lxml import etree

NS = {"w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}

In [251]:
docx_path = "Questionnaire ESEA2026_v1.6_adapte.docx"

with zipfile.ZipFile(docx_path) as z:
    xml = z.read("word/document.xml")
tree = etree.XML(xml)

In [252]:
values = {
    'SIEGE_NUM_VOIE_C' : '4',
    'SIEGE_CODE_COM_C' : '31320',
    'SIEGE_LIB_COM_C' : 'Castanet',
    'SIEGE_LIB_REG_C' : 'Occitanie',
    'REP_NOM_1' : 'Ambio',
    'BIO_11110000' :  'Oui',
    'BIO_11400000' : 'Non'
}

In [253]:
def FindandReplace(find_word, replace_word):

   for p in tree.xpath("//w:p", namespaces=NS):
        texts = p.xpath(".//w:t", namespaces=NS)
        full_text = "".join(t.text or "" for t in texts)

        if find_word not in full_text:
            continue

        new_text = full_text.replace(find_word, ' ' + replace_word)

    # Clear existing runs
        for t in texts:
            t.text = ""

    # Put replaced text into the first run
        texts[0].text = new_text 

In [254]:
for find_word, replace_word in values.items():
    FindandReplace(find_word, replace_word)

In [256]:
with zipfile.ZipFile('Questionnaire ESEA2026_v1.6_adapte.docx') as z:
    files = {name: z.read(name) for name in z.namelist()}

files["word/document.xml"] = etree.tostring(
    tree, xml_declaration=True, encoding="UTF-8", standalone="yes"
)

with zipfile.ZipFile("Questionnaire ESEA2026_v1.6_complete.docx", "w") as z:
    for name, data in files.items():
        z.writestr(name, data)